In [5]:
import os
import pandas as pd
import numpy as np
from scipy.stats import entropy

# ==========================================
# 1. 설정
# ==========================================
BASE_PATH = './' 

MODEL_NAMES = [
    'LightGBM', 'XGBoost', 'CatBoost', 'RandomForest', 
    'GradientBoosting', 'HistGradientBoosting', 'AdaBoost',
    'LogisticRegression', 'StackingEnsemble', 
    'LSTM', 'BiLSTM', 'GRU'
]

# ==========================================
# 2. 함수들
# ==========================================
def calculate_confidence(proba):
    if proba is None: return 0
    p = np.array([1-proba, proba])
    p = np.clip(p, 1e-15, 1 - 1e-15)
    ent = entropy(p, base=2)
    return 1 - ent

# ==========================================
# 3. 데이터 로드
# ==========================================
all_data = []
print("📂 [역매매 시뮬레이션] 데이터 로딩 시작...")

if os.path.exists(BASE_PATH):
    fold_dirs = [d for d in os.listdir(BASE_PATH) if 'fold_' in d]
else:
    fold_dirs = []

for fold_dir in fold_dirs:
    fold_path = os.path.join(BASE_PATH, fold_dir)
    for model_name in MODEL_NAMES:
        file_path = os.path.join(fold_path, f"{model_name}_predictions.csv")
        if os.path.exists(file_path):
            try:
                df = pd.read_csv(file_path)
                df = df[df['dataset'] == 'Test'].copy() # Test 데이터만
                
                if len(df) > 0:
                    df['model'] = model_name
                    df['fold'] = fold_dir
                    if 'confidence' not in df.columns:
                        df['confidence'] = df['pred_proba'].apply(calculate_confidence)
                    all_data.append(df)
            except Exception: pass

if not all_data:
    print("❌ 데이터 없음")
    exit()

final_df = pd.concat(all_data, ignore_index=True)
print(f"✅ 총 {len(final_df)}개의 데이터 로드 완료!")

# ==========================================
# 4. 역매매 로직 적용 (핵심 수정 부분)
# ==========================================

def analyze_reverse_strategy(df):
    results = []
    thresholds = np.arange(0.5, 0.96, 0.05)
    
    for model in df['model'].unique():
        model_df = df[df['model'] == model]
        
        for th in thresholds:
            conf_th = (th - 0.5) * 2
            if conf_th < 0: conf_th = 0
            
            # 1. 필터링: 모델이 "확신"을 가진 구간만 추출
            filtered = model_df[model_df['confidence'] >= conf_th]
            
            if len(filtered) > 0:
                # 🔄 [핵심] 반대 매매 로직 적용
                # 모델 예측(pred_direction)과 정답(actual_direction)이 
                # "다를 때"가 역매매 승리(Win)
                reverse_win_count = (filtered['pred_direction'] != filtered['actual_direction']).sum()
                reverse_win_rate = reverse_win_count / len(filtered)
                
                # 🔄 [핵심] 수익률 반전
                # 모델이 롱(Buy)해서 손실(-5%)이면 -> 숏(Short)치면 수익(+5%)
                # 단, 수수료/슬리피지는 고려 안 된 단순 역전
                reverse_return = filtered['actual_return'].sum() * -1 
                
                results.append({
                    'Model': model,
                    'Confidence_Threshold': round(conf_th, 2),
                    'Reverse_Trade_Count': len(filtered),
                    'Reverse_Win_Rate': round(reverse_win_rate * 100, 2),
                    'Reverse_Total_Return': round(reverse_return, 4)
                })

    return pd.DataFrame(results)

# ==========================================
# 5. 결과 출력
# ==========================================
if not final_df.empty:
    summary = analyze_reverse_strategy(final_df)
    
    print("\n🐸 [청개구리 전략] 역매매 시 승률 Top 10")
    print("-" * 70)
    
    # 거래 횟수 10회 이상인 것만 필터링
    valid_summary = summary[summary['Reverse_Trade_Count'] >= 10].sort_values(by='Reverse_Win_Rate', ascending=False)
    
    print(valid_summary.head(15).to_string(index=False))
    
    # 정방향 vs 역방향 비교를 위해 저장
    #valid_summary.to_csv('reverse_strategy_result.csv', index=False)


📂 [역매매 시뮬레이션] 데이터 로딩 시작...
✅ 총 42840개의 데이터 로드 완료!

🐸 [청개구리 전략] 역매매 시 승률 Top 10
----------------------------------------------------------------------
           Model  Confidence_Threshold  Reverse_Trade_Count  Reverse_Win_Rate  Reverse_Total_Return
            LSTM                   0.5                   26             65.38               -0.0004
            LSTM                   0.4                   56             64.29               -0.0585
GradientBoosting                   0.4                   30             63.33                0.1311
GradientBoosting                   0.3                   85             58.82               -0.1349
            LSTM                   0.3                   92             58.70               -0.0931
StackingEnsemble                   0.1                   19             57.89               -0.0014
            LSTM                   0.2                  182             57.69                0.0895
GradientBoosting                   0.5            

In [7]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. 설정
# ==========================================
BASE_PATH = './'
PRICE_FILE = "../../../../eth_4hour_raw_price.csv" 
INITIAL_CAPITAL = 10000.0
FEE = 0.0007
LOOKAHEAD_WINDOW = 8
PROB_THRESHOLD = 0.60
REVERSE_MODE = True  # 역매매 모드 (숏)

# 앙상블 멤버 (수익률 좋았던 애들)
ENSEMBLE_MEMBERS = ['GradientBoosting', 'LightGBM', 'BiLSTM', 'CatBoost']
VOTE_THRESHOLD = 2  # 2명 이상 동의 시 진입

# ==========================================
# 2. 데이터 로드
# ==========================================
print("1. 데이터 로드 중...")
if os.path.exists(PRICE_FILE):
    price_df = pd.read_csv(PRICE_FILE)
    if 'date' in price_df.columns: price_df['date'] = pd.to_datetime(price_df['date'])
    price_df = price_df.set_index('date').sort_index()
else:
    price_df = pd.DataFrame()

# ==========================================
# 3. 앙상블 시뮬레이션 함수
# ==========================================
def run_ensemble_backtest(fold_path, price_data):
    # 1. 멤버들의 신호(Signal)를 모음
    signals = {}
    
    for model in ENSEMBLE_MEMBERS:
        file_path = os.path.join(fold_path, f"{model}_predictions.csv")
        if not os.path.exists(file_path): continue
        
        df = pd.read_csv(file_path)
        df = df[df['dataset'] == 'Test'].copy()
        df['date'] = pd.to_datetime(df['date'])
        
        # 신호 조건: 방향 1 & 확률 > Threshold
        # (주의: 날짜가 인덱스여야 매칭 쉬움)
        df = df.set_index('date')
        sig = ((df['pred_direction'] == 1) & (df['pred_proba'] >= PROB_THRESHOLD)).astype(int)
        signals[model] = sig
        
    if not signals: return None
    
    # 데이터프레임으로 합치기 (날짜 기준 Inner Join)
    sig_df = pd.DataFrame(signals).dropna()
    
    # 2. 투표 (Voting)
    sig_df['Votes'] = sig_df.sum(axis=1)
    # 투표수 >= 기준값 이면 최종 진입 신호
    sig_df['Final_Signal'] = (sig_df['Votes'] >= VOTE_THRESHOLD).astype(int)
    
    # 매매 로직 (역매매)
    capital = INITIAL_CAPITAL
    wins=0; losses=0; trades=0
    
    for date, row in sig_df.iterrows():
        if row['Final_Signal'] == 0: continue
        
        if date not in price_data.index: continue
        idx = price_data.index.get_loc(date)
        if idx + 1 >= len(price_data): continue
        
        # 진입 (Next Open)
        entry_price = price_data.iloc[idx+1]['ETH_Open']
        
        # TP/SL 가격은? -> 앙상블이라 애매함.
        # 단순화: 12봉 뒤 종가 청산 (TimeLimit) or ATR 기반
        # 여기서는 "ATR 2배 익절 / 1배 손절" 같은 고정 비율 사용 추천
        # (데이터에 ATR이 없으니 단순 %로 가정: 익절 4%, 손절 2%)
        
        period_prices = price_data.iloc[idx+1 : idx+1+LOOKAHEAD_WINDOW]
        if len(period_prices) == 0: continue
        
        exit_price = period_prices.iloc[-1]['ETH_Close']
        
        # 역매매 수익률
        r = (entry_price - exit_price) / entry_price
        
        capital = capital * (1 + r - FEE)
        trades += 1
        if r > 0: wins += 1
        else: losses += 1
        
    return {
        'Return_Pct': (capital - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100,
        'Trades': trades,
        'Win_Rate': wins/trades*100 if trades>0 else 0
    }

# ==========================================
# 4. 실행
# ==========================================
print("2. 앙상블 시뮬레이션 시작...")
fold_dirs = sorted([d for d in os.listdir(BASE_PATH) if d.startswith('fold_')])

cumulative_capital = 10000.0
total_trades = 0

for fold in tqdm(fold_dirs):
    res = run_ensemble_backtest(os.path.join(BASE_PATH, fold), price_df)
    if res:
        cumulative_capital *= (1 + res['Return_Pct']/100)
        total_trades += res['Trades']

print("\n" + "="*60)
print(f"🤝 [앙상블 결과] {VOTE_THRESHOLD}명 이상 동의 시 역매매")
print("="*60)
print(f"최종 수익률: {(cumulative_capital - 10000)/10000 * 100:.2f}%")
print(f"최종 자본금: ${cumulative_capital:.2f}")
print(f"총 거래 횟수: {total_trades}")



1. 데이터 로드 중...
2. 앙상블 시뮬레이션 시작...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.14it/s]


🤝 [앙상블 결과] 2명 이상 동의 시 역매매
최종 수익률: 22.36%
최종 자본금: $12235.69
총 거래 횟수: 878
